In [5]:
import pandas as pd
import numpy as np

In [6]:
#sizes = {'real': {'train': 443757, 'val': 214354}, 'abstract': {'train': 60000, 'val': 30000}}

In [43]:
def get_summary(scenario, _set = 'val'):
    details_train = pd.read_csv('./data/predictions_details_{}_train'.format(scenario), header=None)
    details_train['_set'] = 'train'
    details_val = pd.read_csv('./data/predictions_details_{}_val'.format(scenario), header=None)
    details_val['_set'] = 'val'
    details = pd.concat([details_train, details_val])
    details[2] = details[2].astype(int)
    details.sort_values(by=2, inplace=True)

    prediction = pd.read_csv('./concatenated_data/{}_prediction.csv'.format(scenario))
    prediction.rename(columns={'2049':'q_type', '2048':'q_id'}, inplace=True)
    
    
    prediction['_set'] = details.set_index(2)[['_set']].loc[prediction['q_id'].astype(int)].values

    #prediction['my_acc'] = (prediction.my_predicted == prediction.my_target).astype(int)
    #prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    prediction['my_score'] = prediction.score
    prediction['base_score'] = prediction.base_score

    sset_prediction = prediction[prediction._set == _set]
    #accs = (sset_prediction[['my_acc','base_acc']].sum()/sset_prediction.shape[0]).values
    scores = (sset_prediction[['my_score','base_score']].sum()/sset_prediction.shape[0]).values


    #print('acc', accs, accs[0] > accs[1])
    print('scores', scores, scores[0] > scores[1])


    accurracies = sset_prediction[['q_type', 'my_score','base_score',
                              'sum_top_down', 'sum_bottom_up']].groupby(['q_type']).sum().reset_index()
                                # 'my_acc','base_acc', 
    accurracies = accurracies.set_index('q_type')
    
    
    new_summary = pd.read_csv('./concatenated_data/queries_summary_{}.csv'.format(scenario))
    del new_summary['count_top_down']
    new_summary.rename(columns={'2049':'_q_type'}, inplace=True)
    
    
    new_summary = new_summary[new_summary._q_type.isin(accurracies.index.values)]
    new_summary['q_type'] = accurracies.loc[new_summary._q_type.values].index.values
    #new_summary['my_acc'] = accurracies.loc[new_summary._q_type.values].my_acc.values
    #new_summary['base_acc'] =  accurracies.loc[new_summary._q_type.values].base_acc.values
    new_summary['my_score'] = accurracies.loc[new_summary._q_type.values].my_score.values
    new_summary['base_score'] =  accurracies.loc[new_summary._q_type.values].base_score.values
    val_count = pd.DataFrame(sset_prediction.q_type.value_counts())
    new_summary['_count'] = val_count.loc[new_summary._q_type.values].values
    new_summary['above_median'] = new_summary['_count'] > float(new_summary['_count'].median())
    new_summary['sum_top_down'] = accurracies.loc[new_summary._q_type.values].sum_top_down.values
    new_summary['sum_bottom_up'] =  accurracies.loc[new_summary._q_type.values].sum_bottom_up.values
    new_summary['winner'] = np.array(['top_down',
                                      'bottom_up'])[(new_summary.sum_bottom_up 
                                       > new_summary.sum_top_down).astype(int)]
    _sum = new_summary[['my_score','base_score']].sum() #'my_acc','base_acc',
    
    #print('acc_sum', _sum.values[:2], _sum[0] > _sum[1])#/accurracies.shape[0]
    print('score_sum', _sum.values, _sum[0] > _sum[1])#/accurracies.shape[0]


    assert (new_summary._q_type != new_summary.q_type).sum() == 0
    _sum = (new_summary.sum_top_down + new_summary.sum_bottom_up).values
    new_summary['sum_top_down'] = new_summary['sum_top_down']/_sum
    new_summary['sum_bottom_up'] = new_summary['sum_bottom_up']/_sum
    new_summary['_diff'] = (new_summary.sum_top_down - new_summary.sum_bottom_up)
    del new_summary['diff']
    del new_summary['q_type']
    del new_summary['sum_top_down']
    
    #del new_summary['base_acc']
    #del new_summary['my_acc']



    print(new_summary._count.sum())
    return new_summary

In [47]:
def get_differents(p=1):
    mean_dif = summary._diff.mean()
    std_dif = summary._diff.std()
    data_dif = summary._diff
    data_diff = pd.DataFrame( ((data_dif < mean_dif - p * std_dif).astype(int) + 
                             (data_dif > mean_dif + p * std_dif).astype(int)) > 0 )
    significantly_different = summary.loc[data_diff[data_diff._diff].index]
    print(data_diff.shape)
    different = significantly_different[significantly_different._count>2].sort_values(by='winner')
    

    different['diff_score'] = different.my_score - different.base_score
    different = different[different.diff_score!=0]

    print(np.round(float(different.shape[0]) / data_diff.shape[0] * 100, 3),'% query_types')
    print(np.round(float(different._count.sum())/ summary._count.sum() * 100, 3),'% queries')
    
    vals = different[['my_score','base_score']].sum().values
    print(vals, vals[0]>vals[1])
    print(different.diff_score.sum(), 'abs diff')
    
    print(different.shape[0],'len')
    different.sort_values(by=['winner','diff_score'], inplace=True, ascending=[True, False])
    return different

In [52]:
scenario = 'real'
summary = get_summary(scenario, _set='val')
#p = 1.31 p = 1.38 p = 1
print('\n')
different = get_differents(1)

('scores', array([ 0.63612855,  0.63371339]), True)
('score_sum', array([ 136356.7,  135839. ]), True)
214354


(317, 1)
(11.041, '% query_types')
(11.644, '% queries')
(array([ 18095.2,  18012.5]), True)
(82.70000000001356, 'abs diff')
(35, 'len')


In [53]:
different

,_q_type,sum_bottom_up,count_bottom_up,_count,over_median,winner,my_score,base_score,above_median,_diff,diff_score
391,what color is the,0.514800,0.5,14073,True,bottom_up,11018.9,10946.8,True,-0.029601,72.1
389,what color are the,0.517062,0.5,3221,True,bottom_up,2426.4,2419.0,True,-0.034123,7.4
485,who many,0.514453,0.5,18,True,bottom_up,7.2,4.2,True,-0.028907,3.0
253,name on,0.518880,0.5,7,True,bottom_up,6.0,3.6,False,-0.037761,2.4
501,who rides,0.524430,0.5,7,True,bottom_up,5.0,4.0,False,-0.048860,1.0
373,urban or,0.526171,0.5,14,True,bottom_up,14.0,13.0,True,-0.052343,1.0
570,will those,0.524738,0.5,3,True,bottom_up,2.0,1.0,False,-0.049476,1.0
324,sunny or,0.536791,0.5,50,True,bottom_up,42.9,42.4,True,-0.073581,0.5
422,where might,0.521554,0.5,12,True,bottom_up,7.4,7.3,True,-0.043107,0.1
384,what animal is,0.517966,0.5,836,True,bottom_up,620.9,621.0,True,-0.035931,-0.1
